<a href="https://colab.research.google.com/github/ykamen/CS4342/blob/main/hw1part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from google.colab import drive

Python 3.7.13


In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
faces = np.load('drive/MyDrive/trainingFaces.npy', 'r')
labels = np.load('drive/MyDrive/trainingLabels.npy', 'r')

In [8]:
fPC(labels, jMap(faces,7)[:,9])

0.4705

In [9]:
print(faces[0][575])

-0.6677621653616024


One predictors comparison to all other pixels

In [16]:
def p13(x,k):
  return np.tile(x,(k,1))

def jMap(trset,p1):
  vector = trset[:,p1]
  vmaptemp = p13(vector,trset.shape[1])
  vmap = vmaptemp.T
  return vmap > trset
#returns matrix showing the result of comparing pixel 1 to every other pixel
#across all available images. jmap

def fPC (y, yhat):
  return np.mean(y == yhat)

def bestJ(jmap,y):
  pc = 0
  pos = 0
  for i in range(jmap.shape[1]):
    temp = fPC(y, jmap[:,i])
    if(temp > pc):
      pc = temp
      pos = i
  return pc, pos
#returns the best predictor in one family of matrices e.g. best p1 > p2 
#for a single p1.

def bestFirstJ(trset,y):
  pc = 0
  p1 = 0
  p2 = 0
  for i in range(trset.shape[1]):
    temp_pc, temp_p2 = bestJ(jMap(trset,i),y)
    if(temp_pc > pc):
      pc = temp_pc
      p2 = temp_p2
      p1 = i
  return jMap(trset,p1)[:,p2]
#returns true/false values array

def jsAcc(y,js):
  c = 0
  a = np.average(js,axis=1)
  for i in range(y.shape[0]):
    if (a[i] <= .5 and y[i]== False):
      c+=1
    elif (a[i] > .5 and y[i]== True):
      c+=1
  return c/y.shape[0]

def bestJS(jmap,js,y):
  pc = 0
  p2 = 0
  for i in range(jmap.shape[1]):
    temp_js = np.stack((jmap[:,i],js),axis=1)
    temp = jsAcc(y,temp_js)
    if (temp>pc):
      pc = temp
      p2 = i
  return pc, p2

def bestNextJ(trset,js,y):
  pc = 0                                                                #starting acc. has to be at least current acc.
  p1 = 0
  p2 = 0
  for i in range(trset.shape[1]):                                                 #for all possible p1 values
    temp_pc, temp_p2 = bestJS(jMap(trset,i),js,y)
    if(temp_pc>pc):
      pc = temp_pc
      p2 = temp_p2
      p1 = i
  return jMap(trset,p1)[:,p2]

In [17]:
bestNextJ(faces,bestFirstJ(faces,labels),labels)

array([ True,  True,  True, ...,  True,  True,  True])